<a href="https://colab.research.google.com/github/123Vind/Diabetis-Predictor/blob/master/diabetisNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
!pip install imbalanced-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [34]:
import tensorflow as tf
import numpy as np
import pandas as pd

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks


In [35]:
#importing dataset from google drive
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
diabetis =  pd.read_csv('/content/drive/My Drive/Colab Notebooks/diabetis.csv',header = None)

In [37]:
diabetis.head

<bound method NDFrame.head of       0    1   2   3    4     5      6   7  8
0     6  148  72  35    0  33.6  0.627  50  1
1     1   85  66  29    0  26.6  0.351  31  0
2     8  183  64   0    0  23.3  0.672  32  1
3     1   89  66  23   94  28.1  0.167  21  0
4     0  137  40  35  168  43.1  2.288  33  1
..   ..  ...  ..  ..  ...   ...    ...  .. ..
763  10  101  76  48  180  32.9  0.171  63  0
764   2  122  70  27    0  36.8  0.340  27  0
765   5  121  72  23  112  26.2  0.245  30  0
766   1  126  60   0    0  30.1  0.349  47  1
767   1   93  70  31    0  30.4  0.315  23  0

[768 rows x 9 columns]>

In [38]:
X = diabetis.iloc[:,:-1]
y = diabetis.iloc[:,-1]


In [39]:
#scaling the training data
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)


In [40]:
X

,0,1,2,3,4,5,6,7
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33
...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63
764,2,122,70,27,0,36.8,0.340,27
765,5,121,72,23,112,26.2,0.245,30
766,1,126,60,0,0,30.1,0.349,47


In [41]:

y.value_counts()


0    500
1    268
Name: 8, dtype: int64

In [42]:
#to get rid of the unbalanced class
oversample = SMOTE()
X_res, y_res = oversample.fit_resample(X_scaled, y)
#tl = TomekLinks(sampling_strategy=0.5)
#X_res, y_res = tl.fit_resample(X,y)

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X_res,y_res,random_state = 2,test_size = 0.25,stratify = y_res)

In [44]:
X_train

array([[0.17647059, 0.71356784, 0.6557377 , ..., 0.4828614 , 0.05209223,
        0.7       ],
       [0.41176471, 0.71356784, 0.73770492, ..., 0.45305514, 0.02134927,
        0.36666667],
       [0.35294118, 0.67336683, 0.6557377 , ..., 0.68852459, 0.06831768,
        0.41666667],
       ...,
       [0.11764706, 0.50753769, 0.47540984, ..., 0.32488823, 0.03287788,
        0.01666667],
       [0.29411765, 0.49748744, 0.60655738, ..., 0.43219076, 0.05337319,
        0.18333333],
       [0.1137984 , 0.72263175, 0.47594612, ..., 0.4724015 , 0.14844745,
        0.06448576]])

In [45]:
y_res.value_counts()

1    500
0    500
Name: 8, dtype: int64

In [46]:
y_train.shape

(750,)

In [47]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
num_classes = 2

In [48]:
X_train.shape

(750, 8)

In [49]:
def build_model():
  model = Sequential()
  model.add(Dense(6,input_dim=8,activation='relu'))
  model.add(Dense(8,activation='relu'))
  model.add(Dense(12,activation='relu'))
  model.add(Dense(10,activation='relu'))
  model.add(Dense(num_classes,activation='sigmoid'))
  model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
  return model

In [50]:
model = build_model()

In [51]:
y_train.shape

(750, 2)

In [52]:
y_train

array([[1., 0.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [0., 1.]], dtype=float32)

In [53]:
model.summary()
sample_weight = np.ones(shape=(len(y_train),2))
#sample_weight.shape
sample_weight[y_train==[0.,1.] ] = 0.5

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 6)                 54        
                                                                 
 dense_6 (Dense)             (None, 8)                 56        
                                                                 
 dense_7 (Dense)             (None, 12)                108       
                                                                 
 dense_8 (Dense)             (None, 10)                130       
                                                                 
 dense_9 (Dense)             (None, 2)                 22        
                                                                 
Total params: 370
Trainable params: 370
Non-trainable params: 0
_________________________________________________________________


In [54]:
model.fit(X_train,y_train,epochs=500,verbose=2)

Epoch 1/500
24/24 - 1s - loss: 0.6890 - accuracy: 0.5813 - 550ms/epoch - 23ms/step
Epoch 2/500
24/24 - 0s - loss: 0.6852 - accuracy: 0.5013 - 56ms/epoch - 2ms/step
Epoch 3/500
24/24 - 0s - loss: 0.6813 - accuracy: 0.5040 - 52ms/epoch - 2ms/step
Epoch 4/500
24/24 - 0s - loss: 0.6772 - accuracy: 0.5160 - 58ms/epoch - 2ms/step
Epoch 5/500
24/24 - 0s - loss: 0.6697 - accuracy: 0.6320 - 59ms/epoch - 2ms/step
Epoch 6/500
24/24 - 0s - loss: 0.6594 - accuracy: 0.6453 - 62ms/epoch - 3ms/step
Epoch 7/500
24/24 - 0s - loss: 0.6433 - accuracy: 0.6880 - 56ms/epoch - 2ms/step
Epoch 8/500
24/24 - 0s - loss: 0.6264 - accuracy: 0.6867 - 60ms/epoch - 3ms/step
Epoch 9/500
24/24 - 0s - loss: 0.6152 - accuracy: 0.6907 - 71ms/epoch - 3ms/step
Epoch 10/500
24/24 - 0s - loss: 0.5979 - accuracy: 0.6933 - 49ms/epoch - 2ms/step
Epoch 11/500
24/24 - 0s - loss: 0.5840 - accuracy: 0.6987 - 40ms/epoch - 2ms/step
Epoch 12/500
24/24 - 0s - loss: 0.5754 - accuracy: 0.7133 - 58ms/epoch - 2ms/step
Epoch 13/500
24/24 - 0s

In [55]:
y_pred = model.predict(X_test)

In [56]:
y_pred

array([[0.99573064, 0.01215196],
       [0.44078764, 0.5391629 ],
       [0.26513267, 0.54661614],
       [0.71276927, 0.40002504],
       [0.77396214, 0.35719162],
       [0.8475793 , 0.2720049 ],
       [0.1688377 , 0.55468667],
       [0.7664468 , 0.43811005],
       [0.37772766, 0.53888726],
       [0.9054865 , 0.19415233],
       [0.07801196, 0.5656351 ],
       [0.49367797, 0.5411719 ],
       [0.7321315 , 0.41680935],
       [0.9482697 , 0.12715551],
       [0.60280067, 0.46766984],
       [0.65482104, 0.47470328],
       [0.5093095 , 0.54036224],
       [0.8264364 , 0.2682882 ],
       [0.23962608, 0.5510312 ],
       [0.5097253 , 0.5375188 ],
       [0.49038556, 0.5417218 ],
       [0.13082811, 0.56745857],
       [0.9109336 , 0.11743605],
       [0.7612655 , 0.5351668 ],
       [0.70052123, 0.39061576],
       [0.847763  , 0.5312581 ],
       [0.96579766, 0.08874896],
       [0.4969907 , 0.53908277],
       [0.13926592, 0.55484927],
       [0.11465824, 0.55895764],
       [0.

In [57]:
y_test

array([[1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.

In [58]:
y_test = np.argmax(y_test,axis = 1)
y_predict = np.argmax(y_pred,axis=1)

In [59]:
y_test.shape

(250,)

In [60]:
y_predict

array([0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0,
       1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0,
       1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1,
       1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 1])

In [61]:
confumat = metrics.confusion_matrix(y_test,y_predict)

In [62]:
confumat

array([[ 88,  37],
       [ 20, 105]])

In [63]:
result = metrics.classification_report(y_test,y_predict)
print()
print('Classification Results')
print(result)


Classification Results
              precision    recall  f1-score   support

           0       0.81      0.70      0.76       125
           1       0.74      0.84      0.79       125

    accuracy                           0.77       250
   macro avg       0.78      0.77      0.77       250
weighted avg       0.78      0.77      0.77       250

